In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.stats
from ranking_utils import scores_better, rankscore_avgtie, greaters, r_names
import pandas as pd


results_dir = Path(Path.cwd(),'L2R_Continous_NPY') ##
if not results_dir.exists():
    ##load and extract data
    !wget https://cloud.imi.uni-luebeck.de/s/cw2m7XHa87piQY2/download -O L2R_Continous_NPY.zip
    !unzip L2R_Continous_NPY.zip
    
num_labels = 9
num_cases = 8
task_name = 'AbdomenMRCT'


# num_labels = 35
# num_cases = 38
# task_name = 'OASIS' ##discard 2020_MICCAI results

discard_events = [] #eg. ['2020_MICCAI', '2022_2023_Continous']
discard_teams = ['2020_MICCAI/initial',
                 '2020_MICCAI/stephanie_haeger_mevis_post',
                 '2020_MICCAI/tony_mok_hkust',
                 '2020_MICCAI/tony_mok_hkust_unsupervised',
                 '2021_MICCAI/LapIRN(unsupervised)',
                 ] #eg. ['2020_MICCAI/Initial', '2022_2023_Continous/Initial']
                 

In [ ]:

results_paths = sorted([x.parent for x in results_dir.glob(f'*/*/{task_name}_dice.npy') if x.is_file()])

###apply filters
results_paths = list(filter(lambda x: x.parent.name not in discard_events and x.parent.name+'/'+x.name not in discard_teams, results_paths))
###move initial to front
initial_index = [i for i, s in enumerate(results_paths) if s.name == 'Initial'][0]
results_paths.insert(0, results_paths.pop(initial_index))

print(*results_paths, sep = "\n")
events, unique_teams = [x.parent.name for x in results_paths],[x.name for x in results_paths]

N = len(unique_teams)






In [ ]:
dice_3 = np.zeros((N,num_cases,num_labels))
hd95_3 = 5*np.ones((N,num_cases,num_labels))
sdlogj_3 = 5*np.ones((N,num_cases))
times_3 = 100*np.ones((N,num_cases))


for i, r_path in enumerate(results_paths):
    team = r_names(r_path.name)
    if Path ( r_path / f'{task_name}_dice.npy').is_file():
        dice = np.load(r_path / f'{task_name}_dice.npy')
        dice[dice != dice] = 0
        dice_3[i,:,:] = dice
    if Path ( r_path / f'{task_name}_hd95.npy').is_file():
        hd95 = np.load(r_path / f'{task_name}_hd95.npy')
        hd95[hd95 != hd95] = 10
        hd95[hd95 == np.Inf] = 10
        hd95_3[i,:,:] = hd95
    if Path ( r_path / f'{task_name}_sd_log_j.npy').is_file() and not team == 'Initial':
        sdlogj = np.load(r_path / f'{task_name}_sd_log_j.npy')
        sdlogj_3[i,:] = sdlogj
    else:
        sdlogj_3[i,:] = 1.5
        
    if Path ( r_path / f'{task_name}_docker_runtimes.npy').is_file() and not team == 'Initial':
        times = np.load(r_path / f'{task_name}_docker_runtimes.npy')
        times_3[i,:] =  times
    else:
        times_3[i,:] = 5000

    # if len (team) < 5:
    #     tab = '\t\t\t\t'
    # elif len (team) < 12:
    #     tab = '\t\t\t'
    # elif len (team) < 24:
    #     tab = '\t\t'
    # else:
    #     tab = '\t'

    # print(i, team, tab + '%0.3f' % dice.mean(), '%0.3f' % hd95.mean(), '%0.3f' % sdlogj_3[i,:].mean(),
    #       '%0.3f' % times_3[i,:].mean())


    


In [ ]:
#robustify
dice0 = dice_3[0]  
dice3_30 = np.zeros((N, round((num_cases*num_labels)*.3)))
idx30 = np.argsort(dice0.reshape(-1))[:dice3_30.shape[1]]
for i in range(N):
    dice3_30[i] = dice_3.reshape(N,-1)[i,idx30]


In [ ]:
rank_all = np.zeros((N,5))
subset = dice_3.reshape(N,-1)
scores = scores_better(subset, N)
rank_dice3a = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,0] = rank_dice3a
subset = dice3_30.reshape(N,-1)
scores = scores_better(subset, N)
rank_dice3b = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,1] = rank_dice3b
subset = hd95_3.reshape(N,-1)
scores = scores_better(-subset, N)
rank_hd3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,2] = rank_hd3
subset = sdlogj_3.reshape(N,-1)
scores = scores_better(-subset, N)
rank_jac3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,3] = rank_jac3
subset = times_3.reshape(N,-1)
scores = scores_better(-subset, N)
rank_time3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,4] = rank_time3

In [ ]:
rank3 = np.power(np.prod(rank_all[:,:],axis=1),.2)
all_rank = np.power(np.prod(rank_all,axis=1),1/5)

In [ ]:
df = pd.DataFrame(columns=['num', 'team', 'Dice', 'Dice_30', 'HD95', 'sdLogJ', 'rank'])
rank3_withoutRUNTIME = np.power(np.prod(rank_all[:,:4],axis=1),.25)
all_rank_withoutRUNTIME = np.power(np.prod(rank_all[:,:4],axis=1),1/4)
idx2 = np.argsort(-all_rank_withoutRUNTIME)


print('WITHOUT RUNTIME')
print('#','team','\t\t\t','Dice\t','Dice30\t','HD95\t',' sdLogJ','rank')


for i in range(N):
    df.loc[i,'rank'] = all_rank_withoutRUNTIME[i]
    team = r_names(unique_teams[idx2[i]])
    new = ''
    if events[idx2[i]] == '2023_MICCAI':
        new = '> '
    tab = '\t\t\t'
    d0 = 5
    d1 = 6
    if(len(team)>=d0):
        tab = '\t\t'
    if(len(team)>d0+d1):
        tab = '\t'
    
    if(len(team)>=d0+d1*2):
        tab = ''
    #if(len(team)>=d0+d1*3):
    #    tab = ''
    print(i+1,new + team,tab,'%0.3f'%dice_3[idx2[i]].mean(),'\t','%0.3f'%dice3_30[idx2[i]].mean(),'\t','%0.3f'%hd95_3[idx2[i]].mean(),' ','%0.3f'%sdlogj_3[idx2[i]].mean(),' ','%0.3f'%rank3_withoutRUNTIME[idx2[i]])
    df.loc[i]= [i+1, team, dice_3[idx2[i]].mean(), dice3_30[idx2[i]].mean(), hd95_3[idx2[i]].mean(), sdlogj_3[idx2[i]].mean(), rank3_withoutRUNTIME[idx2[i]]]

In [ ]:
print(df)

In [ ]:
idx2 = np.argsort(-all_rank)
print('WITH RUNTIME')
print('#','team','\t\t\t','Dice\t','Dice30\t','HD95\t ','sdLogJ ','runtime\t','rank')
for i in range(N):
    team = r_names(unique_teams[idx2[i]])
    new = ''
    if events[idx2[i]] == '2023_MICCAI':
        new = '> '
    tab = '\t\t\t'
    d0 = 5
    d1 = 6
    if(len(team)>=d0):
        tab = '\t\t'
    if(len(team)>d0+d1):
        tab = '\t'
    if(len(team)>=d0+d1*2):
        tab = ''
    #if(len(team)>=d0+d1*3):
    #    tab = ''
    print(i+1,new+team,tab,'%0.3f'%dice_3[idx2[i]].mean(),'\t','%0.3f'%dice3_30[idx2[i]].mean(),'\t','%0.3f'%hd95_3[idx2[i]].mean(),' ','%0.3f'%sdlogj_3[idx2[i]].mean(),' ','%0.4f'%times_3[idx2[i]].mean(),'\t','%0.3f'%rank3_withoutRUNTIME[idx2[i]])


In [ ]:
print(df)
df.to_csv('AbdomenMRCT_ranking.csv', index=False)